<a href="https://colab.research.google.com/github/11doris/jazz-maestro/blob/colab_word_embeddings/tune_similarity_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall gensim -y

Found existing installation: gensim 3.6.0
Uninstalling gensim-3.6.0:
  Successfully uninstalled gensim-3.6.0


In [1]:
!pip install gensim

In [2]:
import gensim
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pprint
import pandas as pd
import numpy as np
from collections import Counter
import plotly.express as px
from tqdm import tqdm 
from gensim.models.doc2vec import Doc2Vec
import pickle
import os

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
print(gensim.__version__)

4.1.2


# Initialization

## Download the Data

Major Triads, Minor Triads and Dom7 chords:

In [5]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd' -O input_seq.txt

--2021-11-13 18:55:13--  https://docs.google.com/uc?export=download&id=13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd
Resolving docs.google.com (docs.google.com)... 172.217.5.206, 2607:f8b0:4007:814::200e
Connecting to docs.google.com (docs.google.com)|172.217.5.206|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/holbt9lnghfl5e44pc8ae3oghi7muppm/1636829700000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download [following]
--2021-11-13 18:55:14--  https://doc-08-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/holbt9lnghfl5e44pc8ae3oghi7muppm/1636829700000/14329102864480165501/*/13WKnD2PIZlJA0TjN9-IbFpD-M7eUCgrd?e=download
Resolving doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleusercontent.com)... 142.250.68.1, 2607:f8b0:4007:80f::2001
Connecting to doc-08-4c-docs.googleusercontent.com (doc-08-4c-docs.googleuse

Meta Data:

In [6]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q' -O input_meta.csv

--2021-11-13 18:55:14--  https://docs.google.com/uc?export=download&id=1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q
Resolving docs.google.com (docs.google.com)... 142.250.72.142, 2607:f8b0:4007:80d::200e
Connecting to docs.google.com (docs.google.com)|142.250.72.142|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2ffu1p8cmlin53r0m8rvgul468nltdm9/1636829700000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download [following]
--2021-11-13 18:55:15--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/2ffu1p8cmlin53r0m8rvgul468nltdm9/1636829700000/14329102864480165501/*/1Qs_SNXKBCeHOZO4VTwIPLR6OsPzEjk6q?e=download
Resolving doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleusercontent.com)... 142.250.68.1, 2607:f8b0:4007:80f::2001
Connecting to doc-14-4c-docs.googleusercontent.com (doc-14-4c-docs.googleu

In [7]:
path_to_file = '/content/input_seq.txt'

with open(path_to_file) as f: 
  lines = f.read().splitlines()

In [8]:
data = [line.split(' ') for line in lines]

### Meta Data

In [9]:
meta = pd.read_csv('input_meta.csv', sep='\t')
meta.columns

Index(['id', 'file_name', 'title', 'composer', 'year', 'year_truncated',
       'tonality', 'tune_key', 'tune_mode', 'structure', 'num_bars',
       'time_signature', 'cycle_fifths_order', 'style', 'musicbrainz_id',
       'wikidata_id', 'wikidata_allmusic', 'wiki_link', 'wikidata_description',
       'lyricist'],
      dtype='object')

In [10]:
titles = meta.loc[:, ['id', 'title', 'tune_mode']].drop_duplicates()
titles.set_index('id', inplace=True)

In [11]:
assert len(data) == len(titles)
print(len(data))

1812


In [12]:
titles_dict = titles.to_dict()
index_to_title = titles_dict['title']
index_to_title[1170]

'These Foolish Things'

In [13]:
title_to_index = {value: index for index, value in index_to_title.items()}
title_to_index['These Foolish Things']

1170

# Visualize Chord N-Grams



In [14]:
def ngrams(tokens, n=2, sep='-'):
    return [sep.join(ngram) for ngram in zip(*[tokens[i:] for i in range(n)])]

In [15]:
def list_to_ngrams(data, n=2, sep='-'):
  out = []
  for row in data:
    out.append(ngrams(row, n, sep))

  return out

In [16]:
data_ngrams = list_to_ngrams(data, 3)

In [17]:
def raw_chords_to_df(tunes):
  tunes_chords = [item for tune in tunes for item in tune]
  counts = Counter(tunes_chords)
  df = pd.DataFrame(counts.items(),
                    columns=['chord', 'count']).sort_values(by='count', ascending=False)

  return df

In [18]:
df_chords = raw_chords_to_df(data_ngrams)
df_chords.head(10)

,chord,count
14,Dm7-G7-C,3344
75,G7-C-C,1630
132,A7-Dm7-G7,1506
37,Em7-A7-Dm7,1132
410,C-C-C,1059
179,D7-G7-C,838
85,G7-G7-C,826
71,Am7-Dm7-G7,663
74,C-G7-C,638
6,Gm7-C7-F,615


In [19]:
df_chords = pd.DataFrame.from_dict(df_chords)
df_chords.sort_values(by=['count'], ascending=False, inplace=True)
df_chords_top = df_chords.query('count > 100')

fig = px.bar(df_chords_top, x='chord', y='count', log_y=True)
fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})
fig.show()

2021-11-13 18:55:19,828 : INFO : NumExpr defaulting to 2 threads.


# Generate Test Data

In [35]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               "Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [36]:
contrafacts_test

[[0, 251],
 [3, 1527],
 [30, 63],
 [80, 1527],
 [104, 1372],
 [190, 1136],
 [211, 1527],
 [222, 1527],
 [260, 1527],
 [262, 1219],
 [303, 894],
 [304, 1527],
 [307, 1136],
 [328, 604],
 [318, 1527],
 [360, 1527],
 [367, 670],
 [387, 1527],
 [404, 1726],
 [408, 168],
 [436, 1527],
 [447, 894],
 [448, 682],
 [474, 1269],
 [588, 1061],
 [590, 1677],
 [599, 1404],
 [677, 236],
 [699, 1660],
 [730, 1527],
 [733, 57],
 [761, 752],
 [829, 1527],
 [848, 651],
 [901, 1527],
 [925, 479],
 [940, 1527],
 [977, 352],
 [992, 1527],
 [996, 1527],
 [1007, 1527],
 [1011, 479],
 [1018, 1527],
 [1023, 1527],
 [1032, 1527],
 [1035, 1527],
 [1163, 1527],
 [1208, 651],
 [1304, 1800],
 [1306, 1679],
 [1138, 1515]]

# TF-IDF

## Build Model

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
data_tfidf = [" ".join(i) for i in data_ngrams]
data_tfidf[:5]

['C-Eb7-G# Eb7-G#-B7 G#-B7-E B7-E-G7 E-G7-Gm7 G7-Gm7-C7 Gm7-C7-F C7-F-G#7 F-G#7-C# G#7-C#-E7 C#-E7-Am7 E7-Am7-D7 Am7-D7-Dm7 D7-Dm7-G7 Dm7-G7-C G7-C-Eb7 C-Eb7-G# Eb7-G#-B7 G#-B7-E B7-E-G7 E-G7-Gm7 G7-Gm7-C7 Gm7-C7-F C7-F-Eb7 F-Eb7-G# Eb7-G#-B7 G#-B7-E B7-E-G7 E-G7-C G7-C-Gm7 C-Gm7-C7 Gm7-C7-Bm7 C7-Bm7-E7 Bm7-E7-A E7-A-C7 A-C7-F C7-F-Bbm7 F-Bbm7-Eb7 Bbm7-Eb7-G# Eb7-G#-Dm7 G#-Dm7-G7 Dm7-G7-C G7-C-Eb7 C-Eb7-G# Eb7-G#-B7 G#-B7-E B7-E-G7 E-G7-Gm7 G7-Gm7-C7 Gm7-C7-F C7-F-Eb7 F-Eb7-G# Eb7-G#-B7 G#-B7-E B7-E-G7 E-G7-C',
 'Am7-Am7-Cm7 Am7-Cm7-Cm7 Cm7-Cm7-Eb Cm7-Eb-Eb Eb-Eb-Em7 Eb-Em7-A7 Em7-A7-Dm7 A7-Dm7-Dm7 Dm7-Dm7-Bm7 Dm7-Bm7-Bm7 Bm7-Bm7-Bbm7 Bm7-Bbm7-Bbm7 Bbm7-Bbm7-Fm7 Bbm7-Fm7-Fm7 Fm7-Fm7-E7 Fm7-E7-E7 E7-E7-Fm7 E7-Fm7-Fm7 Fm7-Fm7-C# Fm7-C#-C# C#-C#-Fm7 C#-Fm7-Fm7 Fm7-Fm7-C# Fm7-C#-C#',
 'Am7-C#-Bm7 C#-Bm7-E7 Bm7-E7-Am7 E7-Am7-C# Am7-C#-Bm7 C#-Bm7-E7 Bm7-E7-Cm7 E7-Cm7-F7 Cm7-F7-Bb F7-Bb-G#m7 Bb-G#m7-C#7 G#m7-C#7-F#m7 C#7-F#m7-B7 F#m7-B7-E B7-E-E E-E-E7 E-E7-Am7 E7-Am7-C# Am7-C#-Bm7 C#-Bm7-E7 

In [24]:
tfidf_vectorizer = TfidfVectorizer()
dt = tfidf_vectorizer.fit_transform(data_tfidf)

In [25]:
type(dt)

scipy.sparse.csr.csr_matrix

In [84]:
df_cos = pd.DataFrame(cosine_similarity(tfidf_matrix),
                      columns=titles['title'],
                      index=titles['title']
                      )
df_cos

title,26-2,500 Miles High,502 Blues,52nd Street Theme,9.20 Special,A Ballad,A Beautiful Friendship,A Blossom Fell,A Certain Smile,A Child Is Born,A Felicidade,A Fine Romance,A Flower Is A Lovesome Thing,A Foggy Day,A Ghost Of A Chance,A Kiss To Build A Dream On,A Little Tear,A Lovely Way To Spend An Evening,A Night In Tunisia,A Nightingale Sang In Berkeley Square,A Pretty Girl Is Like A Melody,A Sleepin' Bee,A Smooth One,A Sound For Sore Ears,A Sunday Kind Of Love,A Taste Of Honey,A Time For Love,A Tisket A Tasket,A Weaver Of Dreams,A Wonderful Day Like Today,Ablution,Ac-Cent-Tchu-Ate The Positive,Across The Alley From The Alamo,Adam's Apple,Affirmation,"African Queen, The",Afro Blue,Afro Centric,After You've Gone,After You,...,When It's Sleepy Time Down South,When My Dreamboat Comes Home,When Somebody Thinks You're Wonderful,When The Midnight Choo Choo Leaves For Alabam,When The Moon Comes Over The Mountain,When The Saints Go Marching In,When You're Smiling,Whispering,"White Cliffs Of Dover, The","White Sport Coat And A Pink Carnation, A",Who's Sorry Now,Why Don't You Go Down To New Orleans,Willie The Weeper,Winin' Boy Blues,Wish Me Luck As You Wave Me Goodbye,"Without You For An Inspiration, Dear",Wooden Heart,Working Man Blues,Wrap Your Troubles In Dreams,Yama Yama Man,Yearning,Yellow Dog Blues,"Yes Sir, That's My Baby",Yes! We Have No Bananas,You Always Hurt The One You Love,You Are My Sunshine,You Belong To Me,You Broke Your Promise,You Can Depend On Me,You Meet The Nicest People In Your Dreams,You Must Have Been A Beautiful Baby,"You Tell Me Your Dreams, I'll Tell You Mine",You Were Only Fooling,You're A Real Sweetheart,You're Driving Me Crazy,You're Foolin' Someone,You're Lucky To Me,You're Nobody Till Somebody Loves You,Your Feet's Too Big,Zing Went The Strings Of My Heart
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26-2,1.000000,0.111764,0.213623,0.321292,0.193126,0.247102,0.268104,0.327810,0.314778,0.169535,0.388903,0.259668,0.201880,0.402508,0.386387,0.257777,0.177972,0.467172,0.090736,0.365390,0.718987,0.299512,0.459669,0.338969,0.319770,0.044873,0.303548,0.271070,0.389645,0.330207,0.535083,0.334294,0.334947,0.304290,0.330155,0.054397,0.114383,0.085960,0.289932,0.417576,...,0.407979,0.371576,0.397886,0.300931,0.345253,0.308970,0.145367,0.407089,0.288436,0.140083,0.201541,0.367471,0.177561,0.164976,0.275750,0.300035,0.325527,0.317329,0.274141,0.351894,0.394415,0.271571,0.345896,0.536116,0.162030,0.391773,0.159949,0.155375,0.272049,0.311482,0.252204,0.167375,0.240757,0.362132,0.357766,0.430585,0.215786,0.146579,0.102567,0.115419
500 Miles High,0.111764,1.000000,0.308800,0.111728,0.126774,0.206469,0.135857,0.075321,0.195516,0.134072,0.220334,0.115953,0.017066,0.119592,0.385305,0.355685,0.145446,0.164412,0.143589,0.320076,0.110700,0.160836,0.036370,0.135843,0.397566,0.057898,0.287456,0.131034,0.224478,0.157595,0.266671,0.081658,0.232776,0.052591,0.128760,0.057787,0.220494,0.237518,0.155346,0.262432,...,0.065289,0.069538,0.055706,0.085088,0.032694,0.000000,0.038057,0.053720,0.039654,0.000000,0.107854,0.000000,0.081755,0.047846,0.084146,0.066939,0.000000,0.000000,0.116414,0.016613,0.000000,0.012402,0.000000,0.024022,0.038379,0.000000,0.015242,0.025631,0.114438,0.018898,0.077826,0.015189,0.145656,0.003525,0.024538,0.030911,0.133645,0.113945,0.000000,0.000000
502 Blues,0.213623,0.308800,1.000000,0.168464,0.235336,0.343072,0.115551,0.000000,0.287100,0.359525,0.533636,0.192603,0.000000,0.099193,0.177529,0.105393,0.051666,0.243559,0.277577,0.591927,0.210666,0.280767,0.000000,0.416900,0.082158,0.142315,0.712924,0.037231,0.433704,0.192368,0.384732,0.000000,0.095170,0.066069,0.246530,0.203470,0.532898,0.529578,0.209625,0.213792,...,0.287009,0.000000,0.064644,0.052608,0.069586,0.000000,0.000000,0.129609,0.028994,0.000000,0.195063,0.000000,0.225196,0.060936,0.164679,0.000000,0.000000,0.000000,0.262041,0.057233,0.116145,0.146671,0.000000,0.139404,0.010103,0.000000,0.000000,0.066227,0.04312

## Test of the Contrafacts tunes

In [49]:
t1 = []
t2 = []
simval = []
for tune1, tune2 in contrafacts_test:
  t1.append(index_to_title[tune1])
  t2.append(index_to_title[tune2])
  simval.append(cosine_similarity(tfidf_matrix[tune1], tfidf_matrix[tune2])[0][0])
  

df_eval_tfidf = pd.DataFrame(zip(t1, t2, simval), columns=['tune1', 'tune2', 'cos_sim'])  
df_eval_tfidf

,tune1,tune2,cos_sim
0,26-2,Confirmation,0.574305
1,52nd Street Theme,I Got Rhythm,0.825817
2,Ablution,All The Things You Are,0.879810
3,Anthropology,I Got Rhythm,0.784389
4,Ballade,As Long As I Live,0.647502
5,Bright Mississippi,Sweet Georgia Brown,0.968172
6,C.T.A.,I Got Rhythm,0.704302
7,Celia,I Got Rhythm,0.486457
8,Cottontail,I Got Rhythm,0.912508
9,Countdown,Tune Up,0.712231


# Doc2Vec Embeddings

## Read and Tag Data

In [ ]:
import smart_open

def read_corpus(fname, ngram_size, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = ngrams(line.split(), ngram_size, '-')
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

## Doc2Vec Hyperparameters

In [ ]:
!rm -R models_doc2vec
!rm -R traindata_doc2vec

rm: cannot remove 'models_doc2vec': No such file or directory
rm: cannot remove 'traindata_doc2vec': No such file or directory


In [ ]:
!mkdir models_doc2vec
!mkdir traindata_doc2vec

In [ ]:
model_path = './models_doc2vec'
model_prefix = 'autos'
data_path = './traindata_doc2vec'

param_grid = {'d2v': {'variant': [#'pv-dm', 
                                  'pv-dbow'], 
                      'window': [
                                 #2, 3, 
                                 4, 
                                 #10
                                 ],
                      'sample': [0.1],
                      'ngrams': [
                                 #1, 2, 
                                 3, 
                                 #4
                                 ],
                     }
              }
size = 100
epochs = 50

for algo, params in param_grid.items():
    for variant in params['variant']:
        dm = 1 if variant == 'pv-dm' else 0
        for ngram_size in params['ngrams']:
            
            # prepare data with the provided ngram size
            train_corpus = list(read_corpus(path_to_file, ngram_size=ngram_size))

            # evaluate different window sizes
            for window in params['window']:

                # evaluate the effect of different sample sizes
                for sample in params['sample']:

                    # calculate the model
                    if algo == 'd2v':
                        model = gensim.models.doc2vec.Doc2Vec(vector_size=size, 
                                                              window=window,
                                                              dm = dm,
                                                              min_count=5, 
                                                              sample = sample, # threshold for configuring which higher-frequency words are randomly downsampled
                                                              epochs=epochs,
                                                              )
                        model.build_vocab(train_corpus)
                        model.train(train_corpus, 
                                    total_examples=model.corpus_count, 
                                    epochs=model.epochs)

                    else:
                        model = gensim.models.FastText(data_ngrams, vector_size=size, window=window, sg=sg, epochs=50)

                    file_name = f"{model_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    model.save(file_name + '.model')

                    file_name = f"{data_path}/{model_prefix}_{algo}_{variant}_{ngram_size}_{window}_{sample}"
                    with open(file_name + '.train', 'wb') as f:
                          # Pickle the 'data' dictionary using the highest protocol available.
                          pickle.dump(train_corpus, f, pickle.HIGHEST_PROTOCOL)

2021-11-10 09:06:49,470 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc5,s0.1,t3)', 'datetime': '2021-11-10T09:06:49.470749', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'created'}
2021-11-10 09:06:49,472 : INFO : collecting all words and their counts
2021-11-10 09:06:49,473 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-11-10 09:06:49,506 : INFO : collected 13442 word types and 2046 unique tags from a corpus of 2046 examples and 89538 words
2021-11-10 09:06:49,508 : INFO : Creating a fresh vocabulary
2021-11-10 09:06:49,530 : INFO : Doc2Vec lifecycle event {'msg': 'effective_min_count=5 retains 2440 unique words (18.152060705252193%% of original 13442, drops 11002)', 'datetime': '2021-11-10T09:06:49.530773', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.

In [ ]:
from gensim.models import KeyedVectors

from os import listdir
from os.path import isfile, join

names = [f for f in listdir(model_path) if isfile(join(model_path, f))]
print(names)
models = {}

for name in names:
    file_name = f"{model_path}/{name}"
    print(file_name)
    models[name] = Doc2Vec.load(file_name)

2021-11-10 09:07:01,752 : INFO : loading Doc2Vec object from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model
2021-11-10 09:07:01,760 : INFO : loading dv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model.dv.* with mmap=None
2021-11-10 09:07:01,761 : INFO : loading wv recursively from ./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model.wv.* with mmap=None
2021-11-10 09:07:01,763 : INFO : setting ignored attribute cum_table to None
2021-11-10 09:07:01,803 : INFO : Doc2Vec lifecycle event {'fname': './models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model', 'datetime': '2021-11-10T09:07:01.803874', 'gensim': '4.1.2', 'python': '3.7.12 (default, Sep 10 2021, 00:21:48) \n[GCC 7.5.0]', 'platform': 'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic', 'event': 'loaded'}


['autos_d2v_pv-dbow_3_4_0.1.model']
./models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model


In [ ]:
names = [f for f in listdir(data_path) if isfile(join(data_path, f))]
print(names)
data = {}

for name in names:
    file_name = f"{data_path}/{name}"
    print(file_name)
    with open(file_name, 'rb') as f:
        data[name] = pickle.load(f)

['autos_d2v_pv-dbow_3_4_0.1.train']
./traindata_doc2vec/autos_d2v_pv-dbow_3_4_0.1.train


In [ ]:
for name, m in models.items():
  print(m)

Doc2Vec(dbow,d100,n5,mc5,s0.1,t3)


In [ ]:
def compare_doc_models(models, **kwargs):

    passes = 3

    df = pd.DataFrame()
    names = []
    scores = []
    runs = []

    for p in range(passes):
      print(f"\n*** Run {p}")
      for name, model in models.items():
        ranks = []
        second_ranks = []
        similarities = []

        print(name)

        # load the training corpus data
        train_name = os.path.splitext(name)[0] + '.train'
        train_corpus = data[train_name]
        print(train_corpus[:5])

        # use the model to infer the similarity for a tune
        for doc_id in range(len(train_corpus)):
            inferred_vector = model.infer_vector(train_corpus[doc_id].words)
            sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
            similarities.append(sims)
            rank = [docid for docid, sim in sims].index(doc_id)
            ranks.append(rank)

            second_ranks.append(sims[1])

        counter = Counter(ranks)
        print(counter)

        tunes_sim = counter[0]
        tunes_not_sim = len(train_corpus) - counter[0]
        score = 100 * tunes_not_sim/(tunes_sim+tunes_not_sim)

        print(f"\tTunes similar to themselves in first place: {tunes_sim}")
        print(f"\tTunes not similar in first place: {tunes_not_sim}")
        print(f"\t% of tunes not similar to themselves in first place: {score:.2f}%")

        names.append(name)
        scores.append(score)
        runs.append(p)
      
    df = pd.DataFrame({'name': names, 
                        'score': scores,
                        'pass': runs
                      })

    return df

In [ ]:
df_score = compare_doc_models(models)


*** Run 0
autos_d2v_pv-dbow_3_4_0.1.model
[TaggedDocument(words=['CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-G#7', 'FM7-G#7-C#M7', 'G#7-C#M7-E7', 'C#M7-E7-Am7', 'E7-Am7-D7', 'Am7-D7-Dm7', 'D7-Dm7-G7', 'Dm7-G7-CM7', 'G7-CM7-Eb7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-Eb7', 'FM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-CM7', 'G7-CM7-Gm7', 'CM7-Gm7-C7', 'Gm7-C7-Bm7', 'C7-Bm7-E7', 'Bm7-E7-AM7', 'E7-AM7-C7', 'AM7-C7-FM7', 'C7-FM7-Bbm7', 'FM7-Bbm7-Eb7', 'Bbm7-Eb7-G#M7', 'Eb7-G#M7-Dm7', 'G#M7-Dm7-G7', 'Dm7-G7-CM7', 'G7-CM7-Eb7', 'CM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-Gm7', 'G7-Gm7-C7', 'Gm7-C7-FM7', 'C7-FM7-Eb7', 'FM7-Eb7-G#M7', 'Eb7-G#M7-B7', 'G#M7-B7-EM7', 'B7-EM7-G7', 'EM7-G7-CM7'], tags=[0]), TaggedDocument(words=['Am7-Am7-Cm7', 'Am7-Cm7-Cm7', 'Cm7-Cm7-EbM7', 'Cm7-EbM7-EbM7', 'EbM7-EbM7-Em7b5', 'EbM7-Em7b5-A

In [ ]:
pd.pivot_table(df_score, index = 'name', values = 'score', columns='pass')

pass,0,1,2
name,,,
autos_d2v_pv-dbow_3_4_0.1.model,2.932551,2.932551,2.737048


In [ ]:
import time
t = time.localtime()
timestamp = time.strftime('%Y%m%d_%H%M%S', t)
print(timestamp)

20211110_090824


In [ ]:
score_filename = f'score_{timestamp}.csv'
df_score.to_csv(score_filename)

In [ ]:
 !zip -r models.zip models_doc2vec/ 

  adding: models_doc2vec/ (stored 0%)
  adding: models_doc2vec/autos_d2v_pv-dbow_3_4_0.1.model (deflated 10%)


In [ ]:
from google.colab import files
files.download("models.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download(score_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Assess the best Model

In [ ]:
best_model = 'autos_d2v_pv-dbow_3_4_0.1'

In [ ]:
model = models[f"{best_model}.model"]

#### Assess self-similarity

In [ ]:
ranks = []
second_ranks = []
similarities = []

train_corpus = data[f"{best_model}.train"]

for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    similarities.append(sims)
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [ ]:
import collections

counter = collections.Counter(ranks)
print(counter)

tunes_sim = counter[0]
tunes_not_sim = len(train_corpus) - counter[0]

print()
print(f"Tunes similar to themselves in first place: {tunes_sim}")
print(f"Tunes not similar in first place: {tunes_not_sim}")
print(f"% of tunes not similar to themselves in first place: {100 * tunes_not_sim/(tunes_sim + tunes_not_sim):.2f}%")

Counter({0: 1988, 1: 38, 2: 5, 3: 3, 7: 2, 4: 2, 372: 1, 1730: 1, 824: 1, 2004: 1, 1646: 1, 5: 1, 157: 1, 1522: 1})

Tunes similar to themselves in first place: 1988
Tunes not similar in first place: 58
% of tunes not similar to themselves in first place: 2.83%


In [ ]:
print('Tunes that are similar to other tunes before they are similar to themselves:')
for tune_index, value in enumerate(ranks):
  if value > 0:
    print(f"'{titles.iloc[tune_index, 0]}': {value}")

Tunes that are similar to other tunes before they are similar to themselves:
'Anthropology': 7
'Arise, Her Eyes': 372
'Bags' Groove': 1
'Basin Street Blues': 1
'Bessie's Blues': 1
'Billie's Bounce': 3
'Blue Monk': 2
'Blue Room, The': 1
'Blue Skies': 1
'Blue Sphere': 1
'Bye Bye Blues': 1
'Creole Love Call': 1
'Cyclic Episode': 2
'Get Happy': 1
'I Can't Give You Anything But Love': 1
'Ko Ko': 1
'Lazy River': 1
'Moose The Mooche': 7
'Mr. P.C.': 1
'No Moe': 3
'Now's The Time': 3
'Oleo': 1
'Onmo': 1730
'Pee Wee': 824
'Pinocchio': 2004
'Re Person I Knew': 1646
'Rosetta': 1
'Salt Peanuts': 2
'Shaw 'Nuff': 5
'So What': 1
'Sorcerer, The': 157
'St. Louis Blues': 2
'Struttin' With Some Barbecue': 1
'Sway': 1
'Sweet Georgia Brown': 1
'Tenor Madness': 1
'Theme, The': 4
'Things Ain't What They Used To Be': 1
'Toy Tune': 1522
'Trane’s Blues': 1
'Walkin'': 4
'You Can Depend On Me': 1
'Birth Of The Blues, The': 1
'C'est Si Bon': 1
'Five Foot Two': 1
'Honeysuckle Rose': 1
'I Got Rhythm': 1
'I Scream, Yo

#### Determine Similarity of Tunes

In [ ]:
assert(len(model.dv.vectors == len(list(title_to_index.keys()))))

In [ ]:
tunes_eval_list = [
  'These Foolish Things', 
  'Blue Moon',
  'Hundred Years From Today, A',
  "All God's Chillun Got Rhythm",
  'I Got Rhythm',
  'Bye Bye Blackbird',
  'Old Fashioned Love',
  'Dinah',
  'Honeysuckle Rose'
]

In [ ]:
for tune in tunes_eval_list:
  sim = model.dv.most_similar(title_to_index[tune])

  print()
  print(f"'{tune}' is most similar to:")
  for index, value in sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')



'These Foolish Things' is most similar to:
	0.748: Isn't It A Pity
	0.745: Mimi
	0.703: It's A Lovely Day Today
	0.702: A Foggy Day
	0.700: They Can't Take That Away From Me
	0.698: Long Ago And Far Away
	0.687: Heart And Soul
	0.681: Star-Crossed Lovers, The
	0.680: At Last
	0.679: Wouldn't It Be Loverly

'Blue Moon' is most similar to:
	0.724: Flamingo
	0.722: At Last
	0.709: My Heart Stood Still
	0.707: Jeepers Creepers
	0.687: Sophisticated Lady
	0.687: Teach Me Tonight
	0.682: It Could Happen To You
	0.680: Long Ago And Far Away
	0.679: Misty
	0.676: Here's That Rainy Day

'Hundred Years From Today, A' is most similar to:
	0.636: Chicken
	0.617: Love Is The Sweetest Thing
	0.595: I May Be Wrong (But I Think You're Wonderful)
	0.591: Nightingale Sang In Berkeley Square , A
	0.589: When Somebody Thinks You're Wonderful
	0.583: Save It Pretty Mama
	0.580: Aged And Mellow Blues
	0.578: White Cliffs Of Dover, The
	0.563: Float Me Down The River
	0.562: That's My Weakness Now

'All God

#### Evaluate Similarity for Jazz Contrafacts

A contrafact is a musical composition built using the chord progression of a pre-existing song, but with a new melody and arrangement. Typically the original tune's progression and song form will be reused but occasionally just a section will be reused in the new composition.

https://en.wikipedia.org/wiki/List_of_jazz_contrafacts


In [ ]:
contrafacts = {
               "26-2": "Confirmation",
               "52nd Street Theme": "I Got Rhythm",
               "Ablution": "All The Things You Are",
               "Anthropology": "I Got Rhythm",
               "Ballade": "As Long As I Live",
               "Bright Mississippi": "Sweet Georgia Brown",
               "C.T.A.": "I Got Rhythm",
               "Celia": "I Got Rhythm",
               "Cottontail": "I Got Rhythm",
               "Countdown": "Tune Up",
               "Dewey Square": "Oh, Lady Be Good",
               "Dexterity": "I Got Rhythm",
               "Dig": "Sweet Georgia Brown",
               "Donna Lee": "Indiana (Back Home Again In)",
               "Don't Be That Way": "I Got Rhythm",
               "Eternal Triangle": "I Got Rhythm",
               "Evidence": "Just You, Just Me",
               "Flintstones": "I Got Rhythm",
               "Four On Six": "Summertime",
               "Freight Train": "Blues For Alice",
               "Good Bait": "I Got Rhythm",
               "Hackensack": "Oh, Lady Be Good",
               "Half Nelson": "Lady Bird",
               "Hot House": "What Is This Thing Called Love",
               "Impressions": "So What",
               "In A Mellow Tone (In A Mellotone)": "Rose Room",
               "In Walked Bud": "Blue Skies",
               "Ko Ko": "Cherokee",
               "Lennie's Pennies": "Pennies From Heaven",   ## Lennie's Pennies is in minor and therefore transposed to Amin... not possible to recognize like that
               # "Let's Call This": "Honeysuckle Rose",
               "Little Rootie Tootie": "I Got Rhythm",
               "Little Willie Leaps": "All God's Chillun Got Rhythm",
               "Lullaby Of Birdland": "Love Me Or Leave Me",
               "Moose The Mooche": "I Got Rhythm",
               "My Little Suede Shoes": "Jeepers Creepers",
               "Oleo": "I Got Rhythm",
               "Ornithology": "How High The Moon",
               "Passport": "I Got Rhythm",
               "Quasimodo (Theme)": "Embraceable You",
               "Rhythm-a-ning": "I Got Rhythm",
               "Room 608": "I Got Rhythm",
               "Salt Peanuts": "I Got Rhythm",
               "Satellite": "How High The Moon",
               "Scrapple From The Apple": "Honeysuckle Rose", # A section
               "Scrapple From The Apple": "I Got Rhythm", # B section
               "Segment": "I Got Rhythm",
               "Seven Come Eleven": "I Got Rhythm",
               "Shaw 'Nuff": "I Got Rhythm",
               "Theme, The": "I Got Rhythm",
               "Tour De Force": "Jeepers Creepers",
               "Wow": "You Can Depend On Me",
               "Yardbird Suite": "Rosetta",
               # following tunes are not from wikipedia:
               "Sweet Sue, Just You": "Honeysuckle Rose",  # A section
               # "All Of Me": "Pennies From Heaven", # bars 25-28 of All of Me are same as bars 17-20 of Pennies From Heaven, but different key!
}

contrafacts_test = []
for reference, compare in contrafacts.items():
  contrafacts_test.append([title_to_index[reference], title_to_index[compare]])
  

In [ ]:
most_sim = model.dv.most_similar(1)
for i in most_sim:
  if 247 == i[0]:
    print("found!")

found!


In [ ]:
similarity_found = 0

for tune1, tune2 in contrafacts.items():
  index1 = title_to_index[tune1]
  index2 = title_to_index[tune2]

  sim = model.dv.similarity(index1, index2)
  most_sim = model.dv.most_similar(index1, topn=5)
  
  # check whether the index2 was found among the topn similar tunes
  for i in most_sim:
    if index2 == i[0]:
      similarity_found += 1

  print()
  print(f"'{tune1}' to '{tune2}': {sim}")
  print(f"'{tune1}' is most similar to:")
  for index, value in most_sim:
    print(f'\t{value:.3f}: {index_to_title[index]}')


print()
num_tunes = len(contrafacts)
print(f"Number of tunes tested: {num_tunes}")
print(f"Number of similarities found: {similarity_found}")
print(f"Result: {100 * similarity_found / num_tunes}%")


'26-2' to 'Confirmation': 0.5461316704750061
'26-2' is most similar to:
	0.805: Giant Steps
	0.785: Countdown
	0.779: Satellite
	0.734: I Won't Dance
	0.729: Lady Bird

'52nd Street Theme' to 'I Got Rhythm': 0.6068105101585388
'52nd Street Theme' is most similar to:
	0.816: I've Got My Fingers Crossed
	0.755: Cherry Pink And Apple Blossom White
	0.723: One Sweet Letter From You (alt.)
	0.714: Ol' Man River
	0.713: Stranger On The Shore

'Ablution' to 'All The Things You Are': 0.8708407282829285
'Ablution' is most similar to:
	0.871: All The Things You Are
	0.711: Dolphin, The
	0.709: So Tender
	0.704: Tune Up
	0.698: Unrequited

'Anthropology' to 'I Got Rhythm': 0.6775551438331604
'Anthropology' is most similar to:
	0.998: Passport
	0.998: No Moe
	0.998: Shaw 'Nuff
	0.998: Oleo
	0.998: Theme, The

'Ballade' to 'As Long As I Live': 0.37266454100608826
'Ballade' is most similar to:
	0.723: September In The Rain
	0.677: After You've Gone
	0.639: Crazy Rhythm
	0.613: Under A Blanket Of Bl

In [ ]:
model.dv.similarity(0, 1)

0.35312873

#### Download Model

In [ ]:
from google.colab import files
files.download(f"{model_path}/{best_model}.model")
files.download(f"{data_path}/{best_model}.train")